In [1]:
#-*- coding:utf-8
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Convolution1D, Conv2D, MaxPooling2D, MaxPooling1D, LSTM, Embedding
from keras.optimizers import SGD
import keras.backend as K
import random
import re
import tensorflow as tf
from keras.layers import *
from keras.models import *
from keras.utils import np_utils
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.callbacks import EarlyStopping, ModelCheckpoint, LambdaCallback, LearningRateScheduler
# config_proto = tf.ConfigProto(log_device_placement=0,allow_soft_placement=0)
# config_proto.gpu_options.allow_growth = True
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

Using TensorFlow backend.


In [2]:
X_train = np.load('../../20_Galaxy/ANGRY/2HXT-train.npy')
Y_train = np.load('../../20_Galaxy/ANGRY/2HXT-train_label.npy')

X_valid = np.load('../../20_Galaxy/ANGRY/2HXT-val.npy')
Y_valid = np.load('../../20_Galaxy/ANGRY/2HXT-val_label.npy')

In [3]:
train_data = np.reshape(X_train,(-1,3640,12))
val_data = np.reshape(X_valid,(-1,3640,12))
one_hot = preprocessing.OneHotEncoder(sparse = False)
y_train = one_hot.fit_transform(Y_train)
y_valid = one_hot.fit_transform(Y_valid)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)


def identity_block1(x, nb_filter, kernel_size=5, drop_rate=0.0):
    out1 = Conv1D(nb_filter, kernel_size, padding='same')(x)
    out1 = BatchNormalization()(out1)
    out1 = Activation('relu')(out1)
    out1 = Dropout(drop_rate)(out1)
    out1 = Conv1D(nb_filter, kernel_size, padding='same')(out1)
    out1 = MaxPooling1D(padding='valid')(out1)

    out2 = Conv1D(nb_filter, 1, padding='same')(x)
    out2 = MaxPooling1D(padding='valid')(out2)
    out = add([out1, out2])
    return out

def identity_block2(x, nb_filter, kernel_size=5, drop_rate=0.0):
    out1 = BatchNormalization()(x)
    out1 = Activation('relu')(out1)
    out1 = Dropout(drop_rate)(out1)
    out1 = Conv1D(nb_filter, kernel_size, padding='same')(out1)
    out1 = BatchNormalization()(out1)
    out1 = Activation('relu')(out1)
    out1 = Dropout(drop_rate)(out1)
    out1 = Conv1D(nb_filter, kernel_size, padding='same')(out1)
    out1 = MaxPooling1D(padding='valid')(out1)

    out2 = Conv1D(nb_filter, 1, padding='same')(x)
    out2 = MaxPooling1D(padding='valid')(out2)
    out = add([out1, out2])
    return out
    
def get_model(verbose=False):
    inputs = Input(shape=(3640, 12))
    #start with conv->BN->ReLU
    net = Conv1D(filters=8, kernel_size=5, padding='same')(inputs)
    net = BatchNormalization()(net)
    net = Activation('relu')(net)
    net_branch = net
    net_branch = Conv1D(filters=8, kernel_size=1, padding='same')(net_branch)
    net_branch = MaxPooling1D()(net_branch)
    net = Conv1D(filters=8, kernel_size=9, padding='same')(net)
    net = BatchNormalization()(net)
    net = Activation('relu')(net)
    #net = Dropout(0.1)(net)
    net = Conv1D(filters=8, kernel_size=7, padding='same')(net)
    net = MaxPooling1D()(net)
    net = add([net, net_branch])

    id1 = identity_block1(net,  16, 7, 0.0)
    id2 = identity_block2(id1,  32, 7, 0.25)
    id3 = identity_block2(id2,  64, 5, 0.25)
    id4 = identity_block2(id3,  128, 5, 0.25)
    id5 = identity_block2(id4,  256, 3, 0.25)
    id6 = identity_block2(id5,  256, 3, 0.5)
    id7 = identity_block2(id6,  512, 3, 0.5)
    id8 = identity_block2(id7,  512, 3, 0.5)
#     id7 = identity_block(id6, 128, 3, 0.3)
#     id8 = identity_block(id7, 128, 3, 0.3)
#     id9 = identity_block(id8,  64, 3, 0.4)
#     id10= identity_block(id9,  64, 3, 0.4)
#     id11 =identity_block(id10, 32, 3, 0.5)
#    concat = Concatenate(axis=2)(id8)
    #lstm_out1 = LSTM(64,return_sequences=True)(id8)
#    print lstm_out1
    
  #  flatten1 = Attention_layer()(lstm_out1)
    
    #end with BN->Relu->Dense->Softmax
    outputs = GlobalAveragePooling1D()(id8)
    outputs = BatchNormalization()(outputs)
    outputs = Activation('relu')(outputs)
    outputs = Dense(4, activation='relu')(outputs)
    outputs = Dense(2, activation='softmax')(outputs)
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(loss='sparse_categorical_crossentropy', optimizer = SGD(lr=0.01,decay=1e-5), metrics=['acc'])
    if verbose:
        model.summary()
    return model

In [12]:
model = get_model(verbose=0)
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpoint = ModelCheckpoint('crnn_model.h5', verbose=1, save_best_only=True)
cb = [earlystopper, checkpoint]
results = model.fit(train_data, Y_train, validation_data=(val_data, Y_valid), batch_size=64, epochs=40, shuffle=True, callbacks=cb)


Train on 15658 samples, validate on 4000 samples
Epoch 1/40
15658/15658 [==============================] - 32s 2ms/step - loss: 0.6581 - acc: 0.6075 - val_loss: 0.7018 - val_acc: 0.5152

Epoch 00001: val_loss improved from inf to 0.70182, saving model to crnn_model.h5
Epoch 2/40
15658/15658 [==============================] - 20s 1ms/step - loss: 0.6200 - acc: 0.6625 - val_loss: 0.7212 - val_acc: 0.5195

Epoch 00002: val_loss did not improve from 0.70182
Epoch 3/40
15658/15658 [==============================] - 20s 1ms/step - loss: 0.5458 - acc: 0.7261 - val_loss: 0.8281 - val_acc: 0.5475

Epoch 00003: val_loss did not improve from 0.70182
Epoch 4/40
15658/15658 [==============================] - 19s 1ms/step - loss: 0.5017 - acc: 0.7501 - val_loss: 0.7166 - val_acc: 0.7235

Epoch 00004: val_loss did not improve from 0.70182
Epoch 5/40
15658/15658 [==============================] - 20s 1ms/step - loss: 0.4739 - acc: 0.7698 - val_loss: 0.6715 - val_acc: 0.7050

Epoch 00005: val_loss impr

In [8]:
class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)


def identity_block1(x, nb_filter, kernel_size=50, drop_rate=0.25):
    out1 = Conv1D(nb_filter, kernel_size, padding='same',strides=1)(x)
    out1 = BatchNormalization()(out1)
    out1 = Activation('relu')(out1)
    out1 = Dropout(drop_rate)(out1)
    out1 = Conv1D(nb_filter, kernel_size, padding='same',strides=1)(out1)
    out1 = MaxPooling1D(padding='valid')(out1)

    out2 = Conv1D(nb_filter, 50, padding='same',strides=1)(x)
    out2 = MaxPooling1D(padding='valid')(out2)
    out = add([out1, out2])
    return out

def identity_block2(x, nb_filter, kernel_size=50, drop_rate=0.25):
    out1 = BatchNormalization()(x)
    out1 = Activation('relu')(out1)
    out1 = Dropout(drop_rate)(out1)
    out1 = Conv1D(nb_filter, kernel_size, padding='same',strides=1)(out1)
    out1 = BatchNormalization()(out1)
    out1 = Activation('relu')(out1)
    out1 = Dropout(drop_rate)(out1)
    out1 = Conv1D(nb_filter, kernel_size, padding='same',strides=1)(out1)
    out1 = MaxPooling1D(padding='valid')(out1)

    out2 = Conv1D(nb_filter, 50, padding='same',strides=1)(x)
    out2 = MaxPooling1D(padding='valid')(out2)
    out = add([out1, out2])
    return out
    
def get_model(verbose=False):
    inputs = Input(shape=(3640, 12))
    #start with conv->BN->ReLU
    net = Conv1D(filters=8, kernel_size=5, padding='same',strides=1)(inputs)
    net = BatchNormalization()(net)
    net = Activation('relu')(net)
    net_branch = net
    net_branch = Conv1D(filters=8, kernel_size=1, padding='same',strides=1)(net_branch)
    net_branch = MaxPooling1D()(net_branch)
    net = Conv1D(filters=8, kernel_size=9, padding='same',strides=1)(net)
    net = BatchNormalization()(net)
    net = Activation('relu')(net)
    #net = Dropout(0.1)(net)
    net = Conv1D(filters=8, kernel_size=7, padding='same',strides=1)(net)
    net = MaxPooling1D()(net)
    net = add([net, net_branch])

    id1 = identity_block1(net,  16, 7, 0.0)
    id2 = identity_block2(id1,  32, 7, 0.25)
    id3 = identity_block2(id2,  64, 5, 0.25)
    id4 = identity_block2(id3,  128, 5, 0.25)
    id5 = identity_block2(id4,  256, 3, 0.25)
    id6 = identity_block2(id5,  256, 3, 0.5)
    id7 = identity_block2(id6,  512, 3, 0.5)
    id8 = identity_block2(id7,  512, 3, 0.5)
    #lstm_out1 = LSTM(64,return_sequences=True)(id8)    
  #  flatten1 = Attention_layer()(lstm_out1)    
    #end with BN->Relu->Dense->Softmax
    outputs = GlobalAveragePooling1D()(id8)
    outputs = BatchNormalization()(outputs)
    outputs = Activation('relu')(outputs)
    outputs = Dense(4, activation='relu')(outputs)
    outputs = Dense(2, activation='softmax')(outputs)
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(loss='categorical_crossentropy', optimizer = SGD(lr=0.003,decay=1e-5), metrics=['acc'])
    if verbose:
        model.summary()
    return model

In [12]:
model = get_model(verbose=0)
earlystopper = EarlyStopping(patience=40, verbose=1)
checkpoint = ModelCheckpoint('cr_model2.h5', verbose=1, save_best_only=True)
cb = [earlystopper, checkpoint]
results = model.fit(train_data, y_train, validation_data=(val_data, y_valid), batch_size=128, epochs=200, shuffle=True, callbacks=cb)


Train on 15658 samples, validate on 4000 samples
Epoch 1/200
15658/15658 [==============================] - 38s 2ms/step - loss: 0.6630 - acc: 0.5948 - val_loss: 0.8059 - val_acc: 0.5088

Epoch 00001: val_loss improved from inf to 0.80588, saving model to cr_model2.h5
Epoch 2/200
15658/15658 [==============================] - 26s 2ms/step - loss: 0.6304 - acc: 0.6470 - val_loss: 0.8172 - val_acc: 0.5050

Epoch 00002: val_loss did not improve from 0.80588
Epoch 3/200
15658/15658 [==============================] - 27s 2ms/step - loss: 0.5570 - acc: 0.7173 - val_loss: 0.5963 - val_acc: 0.75423 - acc: 

Epoch 00003: val_loss improved from 0.80588 to 0.59626, saving model to cr_model2.h5
Epoch 4/200
15658/15658 [==============================] - 28s 2ms/step - loss: 0.4969 - acc: 0.7606 - val_loss: 0.5690 - val_acc: 0.8455

Epoch 00004: val_loss improved from 0.59626 to 0.56900, saving model to cr_model2.h5
Epoch 5/200
15658/15658 [==============================] - 28s 2ms/step - loss: 0.46

15658/15658 [==============================] - 28s 2ms/step - loss: 0.3420 - acc: 0.8542 - val_loss: 0.8241 - val_acc: 0.7965

Epoch 00042: val_loss did not improve from 0.45199
Epoch 43/200
15658/15658 [==============================] - 28s 2ms/step - loss: 0.3451 - acc: 0.8511 - val_loss: 0.4636 - val_acc: 0.8350

Epoch 00043: val_loss did not improve from 0.45199
Epoch 44/200
15658/15658 [==============================] - 28s 2ms/step - loss: 0.3408 - acc: 0.8542 - val_loss: 3.1474 - val_acc: 0.5072

Epoch 00044: val_loss did not improve from 0.45199
Epoch 45/200
15658/15658 [==============================] - 28s 2ms/step - loss: 0.3396 - acc: 0.8544 - val_loss: 0.4474 - val_acc: 0.8822

Epoch 00045: val_loss improved from 0.45199 to 0.44742, saving model to cr_model2.h5
Epoch 46/200
15658/15658 [==============================] - 28s 2ms/step - loss: 0.3397 - acc: 0.8542 - val_loss: 0.7003 - val_acc: 0.8377

Epoch 00046: val_loss did not improve from 0.44742
Epoch 47/200
15658/15658

15658/15658 [==============================] - 28s 2ms/step - loss: 0.3119 - acc: 0.8679 - val_loss: 0.5772 - val_acc: 0.8645

Epoch 00084: val_loss did not improve from 0.39827
Epoch 85/200
15658/15658 [==============================] - 28s 2ms/step - loss: 0.3098 - acc: 0.8695 - val_loss: 0.6306 - val_acc: 0.8397

Epoch 00085: val_loss did not improve from 0.39827
Epoch 86/200
15658/15658 [==============================] - 28s 2ms/step - loss: 0.3110 - acc: 0.8687 - val_loss: 0.3651 - val_acc: 0.8900

Epoch 00086: val_loss improved from 0.39827 to 0.36515, saving model to cr_model2.h5
Epoch 87/200
15658/15658 [==============================] - 28s 2ms/step - loss: 0.3106 - acc: 0.8688 - val_loss: 0.4111 - val_acc: 0.8882

Epoch 00087: val_loss did not improve from 0.36515
Epoch 88/200
15658/15658 [==============================] - 28s 2ms/step - loss: 0.3115 - acc: 0.8683 - val_loss: 0.8211 - val_acc: 0.7835

Epoch 00088: val_loss did not improve from 0.36515
Epoch 89/200
15658/15658

15658/15658 [==============================] - 28s 2ms/step - loss: 0.2873 - acc: 0.8798 - val_loss: 0.3846 - val_acc: 0.8958

Epoch 00126: val_loss did not improve from 0.32386
Epoch 127/200
15658/15658 [==============================] - 27s 2ms/step - loss: 0.2888 - acc: 0.8790 - val_loss: 0.9099 - val_acc: 0.7798

Epoch 00127: val_loss did not improve from 0.32386
Epoch 128/200
15658/15658 [==============================] - 28s 2ms/step - loss: 0.2841 - acc: 0.8813 - val_loss: 0.4243 - val_acc: 0.8955 3s - loss: 

Epoch 00128: val_loss did not improve from 0.32386
Epoch 129/200
15658/15658 [==============================] - 28s 2ms/step - loss: 0.2847 - acc: 0.8820 - val_loss: 0.7135 - val_acc: 0.8612

Epoch 00129: val_loss did not improve from 0.32386
Epoch 130/200
15658/15658 [==============================] - 28s 2ms/step - loss: 0.2865 - acc: 0.8804 - val_loss: 0.3896 - val_acc: 0.8635

Epoch 00130: val_loss did not improve from 0.32386
Epoch 131/200
15658/15658 [===============